# HMM

In [49]:
import numpy as np

In [50]:
#number of hidden states: (i:intragenic - p:promoter - c:coding - a:polyA)
n_x = 4 

#names of hidden states
names_x = ["i", "p","c","a"]

#number of revealed states: (A - T - G - C)
n_y = 4 

#names of revealed states
names_y = ["A", "T","G","C"]

#transition matrix for hidden states: model assumption, use P_x[x_prev, x_new]
P_x = np.array([
    [9/10, 1/10, 0, 0],
    [0, 4/5, 1/5, 0],
    [0, 0, 9/10, 1/10],
    [1/6, 0, 0, 5/6]
])

#transition matrix for revealed states: model assumption, use P_x_y[x,y]
P_x_y = np.array([
    [0.25, 0.25, 0.25, 0.25], #intergenic 
    [0.1, 0.1, 0.4, 0.4], #promoter
    [0.1, 0.2, 0.3, 0.4], #coding
    [0.7, 0.1, 0.1, 0.1] #polyA
])


#starting prior
mu_0 = np.array([0.25, 0.25, 0.25, 0.25])

#time 
t = 500

In [51]:
def sample_X():
    X = [np.random.choice(n_x,p=mu_0)]
    for _ in range(1,t):
        x = X[-1]
        X.append(np.random.choice(n_x, p = P_x[x,:]))
    return X

def sample_Y(X):
    Y=[]
    for x in X:
        Y.append(np.random.choice(n_y, p = P_x_y[x,:]))
    return Y

def show_names(X,Y):
    seq_x = ""
    seq_y = ""
    for i in range(len(X)):
        if (i%10==0 and i>0):
            seq_x+= " "
            seq_y+= " "
        seq_x = seq_x + names_x[X[i]]
        seq_y = seq_y + names_y[Y[i]]
        
    print(seq_x)
    print("")
    print(seq_y)

In [52]:
X = sample_X()
Y = sample_Y(X)
show_names(X,Y)

pppppppccc cccaaaaaaa aiippppccc cccccccccc cccccccaaa iiiiiiiiii iiiiiiiiii pccccccaaa aaiiiiiipp ppcccccccc cccaipcccc cccccccccc cccccccccc cccccccccc ccccccccca aaaaaaaaaa aaaaaaaaai iiiiiiiiii iiiiiiiiii iiiiiiiiii iiiiiiiiii iiiiiiiiii iiiiiiiiii iiiiiiippc cccccaaaii pppppppppp pppcaaaipc ccccccccaa aaaaaaaiii pppppppppc cccccccccc cccaaaaaaa aaaaippppp ppccccccaa aaaaaaaaaa aaaaaaaaii pppppppppp ppppppcccc ccaaaiiiii iiiiiiiiii iiiiiiiiii ippppppppp ppcccccccc cccccaaaaa aaiipppppp pccccccccc cccccccccc ccccccaaaa aaaiiiiipp cccccccaaa

GAAGGGACTG TGGAAAGAAA AACCCGGACG ACGCTGCTCA GCAGGACAGA ACACGCAGGC TCAACGACAG CCCGCTGAAA CAGCGAACGG GAATCCGGTG AGCACCCGCT CACTCCCCGG GCCCATGCGT CGTGTGGTTC TTCCCGCGAA CACATTTGAC CCAAAAAGAA CTACATCCAC TAGCGGACGC TGGTTTCCAA GACAACGCTC GGATTTAAGT CGATGCCTTT CTAGGTCCCC GTTGCAAACC GTCCAGCGTG CGGCAAAGCC CGCCCTTCTA ACAACAATGA GCACGTCGTT ATCGCAGTTT TTCAAAAAAA GAAACCGGCC AAGCGGCCAA TATACAAACA AACAAAAATA CCCGGGAGCA CTCACCGCCG TAAAAAGTTG AATCTAAAAT ATACGTGGG

In [53]:
with open("ml_hmm_regions.txt",'r') as f:
    reg = f.read()

reg = reg.replace("\n","")
X_test = [names_x.index(s) for s in reg]

with open("ml_hmm_seq.txt",'r') as f:
    seq = f.read()

seq = seq.replace("\n","")
Y_test = [names_y.index(s) for s in seq]

In [54]:
def forward_pass(Y):
    m_0 = np.sum(mu_0[:]*P_x_y[:, Y[0]]*P_x.T[:,:], axis =1)
    f_messages = [m_0/m_0.sum()]
    for i in range(1,t-1):
        m_prev = f_messages[-1]
        m_new = np.sum(m_prev[:]*P_x_y[:, Y[i]]*P_x.T[:,:], axis =1)
        f_messages.append(m_new/m_new.sum())
    return f_messages

def backward_pass(Y):
    m_0 = np.sum(P_x_y[:,Y[t-1]]*P_x[:,:], axis =1)
    b_messages = [m_0/m_0.sum()]
    for i in range(t-2,0,-1):
        m_prev = b_messages[-1]
        m_new = np.sum(m_prev[:]*P_x_y[:, Y[i]]*P_x[:,:], axis =1)
        b_messages.append(m_new/m_new.sum())
    return b_messages

def posterior(f_messages, b_messages):
    """
        Returns posterior beliefs
    """
    beliefs = []
    belief_0 = mu_0[:]*P_x_y[:,Y[0]]*b_messages[t-2]
    beliefs.append(belief_0/belief_0.sum())
    for i in range(1,t-1):
        belief_i = f_messages[i-1]*P_x_y[:,Y[i]]*b_messages[t-2-i]
        beliefs.append(belief_i/belief_i.sum())
    belief_last = f_messages[t-2]*P_x_y[:,Y[t-1]]
    beliefs.append(belief_last/belief_last.sum())
    #for first el
    return(beliefs)

In [55]:
f_messages = forward_pass(Y_test)
b_messages = backward_pass(Y_test)
posteriors = posterior(f_messages,b_messages)

X_guess_test = [np.argmax(post) for post in posteriors]

np.mean(np.equal(X_test,X_guess_test))

0.674

In [56]:
X = sample_X()
Y = sample_Y(X)
f_messages = forward_pass(Y)
b_messages = backward_pass(Y)

X_guess = [np.argmax(post) for post in posterior(f_messages,b_messages)]

np.mean(np.equal(X,X_guess))

0.614

In [57]:
(500-124)/500

0.752

## Unitary tests

In [40]:
n_x = 2
n_y = 2
P_x = np.array([
    [1, 0],
    [0, 1]
])

P_x_y = np.array([
    [1, 0],
    [1/2, 1/2]
])

X = [0,0,0,0,0]#,0]
Y = [0,0,0,0,0]#,0]

mu_0 = [0.5,0.5]

t = 5

In [41]:
f_messages = forward_pass(Y)
f_messages

[array([0.66666667, 0.33333333]),
 array([0.8, 0.2]),
 array([0.88888889, 0.11111111]),
 array([0.94117647, 0.05882353])]

In [42]:
b_messages = backward_pass(Y)
b_messages

[array([0.66666667, 0.33333333]),
 array([0.8, 0.2]),
 array([0.88888889, 0.11111111]),
 array([0.94117647, 0.05882353])]

In [43]:
posteriors = posterior(f_messages,b_messages)

In [44]:
posteriors

[array([0.96969697, 0.03030303]),
 array([0.96969697, 0.03030303]),
 array([0.96969697, 0.03030303]),
 array([0.96969697, 0.03030303]),
 array([0.96969697, 0.03030303])]

In [45]:
a=1
b=2**5

In [46]:
b/(a+b)

0.9696969696969697

In [ ]:
P_x_y[:,Y[0]]

In [ ]:
b_messages[t-2]

In [ ]:
3/4*(3/4)+1/4*(1/4)

In [ ]:
t

In [ ]:
belief_0 = mu_0[:]*P_x_y[:,Y[0]]*b_messages[t-2]

In [ ]:
belief_0

In [ ]:
a = 3*(27+1)/4/4/10/2

In [ ]:
b = (9+3)/2/4/10/4

In [ ]:
a/(a+b)

In [ ]:
28/40

In [ ]:
f_messages

In [ ]:
a

In [ ]:
9*3/10/4+1/10/4

In [ ]:
9/10/4+3/10/4